In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
from google.colab import drive
import numpy as np
import cv2

#constent stuff
MODEL = "/content/models/glass.tflite"
DATASET_PATH_COLAB = "/content/glass/"
TEST_TEXT = DATASET_PATH_COLAB+"test.txt"

#mounting to drive
drive.mount('/content/gdrive/')

In [2]:
!ln -s "/content/gdrive/My Drive/magshimim/camerona/dataset/glass/" "/content/"
!ln -s "/content/gdrive/My Drive/magshimim/camerona/models/" "/content/"

In [7]:
def getData(file_path):
  """
  This function loads the data from a file
  input: the text file path
  output: x abd y of the data
  """
  #openning the file
  f = open(file_path, 'r')
  data = f.read().split('\n')
  f.close()
  x = []
  y = []
  all = len(data)
  cnt = 1
  for line in data:
    #iterating through the file
    if cnt%250 == 0:
      print("{0}/{1}".format(cnt, all))
    cnt += 1
    line_data = line.split(" ")
    try:
      #loading the image
      img = cv2.imread(DATASET_PATH_COLAB+line_data[0])
      img = cv2.resize(img, (64,64))/255
    except:
      continue
    #adding data
    _y = np.zeros(3)
    _y[int(line_data[1])] = 1
    x.append(img)
    y.append(_y)
  #converting to numoy array
  x = np.array(x)
  y = np.array(y)
  return x,y

In [8]:
class ModelData():
  """
  class that uses the tflite models, its easier to do everything like this
  """
	def __init__(self, model_path):
    """
    This method is the C'tor
    input: model path
    """
		self.model = tf.lite.Interpreter(model_path=model_path)
		self.input_tensor_index = self.model.get_input_details()[0]['index']
		self.output_tensor_index = self.model.get_output_details()[0]['index']
		self.model.allocate_tensors()

	def modelPredict(self, face):
		self.model.set_tensor(self.input_tensor_index, face)
		self.model.invoke()
		prediction = self.model.get_tensor(self.output_tensor_index)
		return prediction

In [9]:
model = ModelData(MODEL)

In [ ]:
test_x, test_y = getData(TEST_TEXT)
test_prediction = []

In [11]:
#getting predictions
for img in test_x:
  img = np.array([img], dtype=np.float32)
  test_prediction.append(model.modelPredict(img))

In [ ]:
#proccesing the data
predictions = test_prediction
p = []
y = []
size = test_y.shape[0]
i = 0
while i < size:
  p.append(predictions[i][0])
  i+=1
predictions = np.array(p)

In [ ]:
#checking accuracy
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(test_y, predictions)
m.result().numpy()